# Data Extraction:

A planilha contém informações detalhadas sobre transações imobiliárias na cidade de São Paulo em 2023, abrangendo aspectos como identificação do imóvel (número de cadastro e matrícula), valores financeiros envolvidos (transação declarada, valor venal), características físicas (áreas do terreno e construída), detalhes do financiamento, informações de registro (cartório e situação cadastral)

Ela pode ser adquirida no seguinte link:
https://www.prefeitura.sp.gov.br/cidade/secretarias/fazenda/acesso_a_informacao/index.php?p=31501

In [2]:
import pandas as pd
import requests
import json
from tqdm import tqdm

Para ler o arquivo talvez seja necessário ter o openpyxl instalado, para isso basta apenas dar um pip install openpyxl no console

In [3]:
# variável com o caminho do arquivo
path = 'GUIAS-DE-ITBI-PAGAS-2023.xlsx'

excel_file = pd.ExcelFile(path)

O arquivo 'GUIAS-DE-ITBI-PAGAS-2023.xlsx' contém várias abas no arquivo de trabalho excel, então primeiramente listamos as abas para selecionar quais vamos trabalhar

In [4]:
sheet_names = excel_file.sheet_names
sheet_names

['JAN-2023',
 'FEV-2023',
 'MAR-2023',
 'ABR-2023',
 'MAI-2023',
 'JUN-2023',
 'JUL-2023',
 'AGO-2023',
 'SET-2023',
 'OUT-2023',
 'NOV-2023',
 'DEZ-2023',
 'LEGENDA',
 'EXPLICAÇÕES',
 'Tabela de USOS',
 'Tabela de PADRÕES']

In [5]:
# Lista com os nomes das abas específicas que queremos trabalhar
sheet_list = ['JAN-2023', 'FEV-2023', 'MAR-2023', 'ABR-2023', 'MAI-2023', 'JUN-2023',
              'JUL-2023', 'AGO-2023', 'SET-2023', 'OUT-2023', 'NOV-2023', 'DEZ-2023']

# Lê todas as planilhas selecionadas em um dicionário
selected_sheets = pd.read_excel(path, sheet_name=sheet_list)

# Concatena as abas selecionadas em um único DataFrame
df_combined = pd.concat(selected_sheets.values(), ignore_index=True)

In [6]:
# configuração para mostrar todas as colunas
pd.set_option('display.max_columns', None)

df_combined.head()

,N° do Cadastro (SQL),Nome do Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza de Transação,Valor de Transação (declarado pelo contribuinte),Data de Transação,Valor Venal de Referência,Proporção Transmitida (%),Valor Venal de Referência (proporcional),Base de Cálculo adotada,Tipo de Financiamento,Valor Financiado,Cartório de Registro,Matrícula do Imóvel,Situação do SQL,Área do Terreno (m2),Testada (m),Fração Ideal,Área Construída (m2),Uso (IPTU),Descrição do uso (IPTU),Padrão (IPTU),Descrição do padrão (IPTU),ACC (IPTU),Unnamed: 28
0,13944000322,AV NORDESTINA,3010,AP 918 TORRE 2,VL CURUCA,NaN,8032000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,15828870.0,0.20,31657.7400,264000.0,1.Sistema Financeiro de Habitação,166320.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0,NaN
1,6726500084,R JURIPIRANGA,70,ANT 16,PARQUE VITORIA,NaN,2269090.0,1.Compra e venda,800000.0,2022-12-23 00:00:00,846885.0,100.00,846885.0000,846885.0,1.Sistema Financeiro de Habitação,400000.00,15º Cartório de Registro de Imóvel,87025,Ativo Predial,210,10.0,1.0,277,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,2009.0,NaN
2,13944000322,AV NORDESTINA,3010,AP 504 SUB 2 BL 2,VL CURUCA,NaN,8032000.0,1.Compra e venda,203700.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,203700.0,1.Sistema Financeiro de Habitação,144059.51,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0,NaN
3,5614900926,R ANTONIO DE BARROS,99999,AP 614 BL 3,TATUAPE,NaN,3401001.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.00,9º Cartório de Registro de Imóvel,336376,Ativo Territorial,1000,20.0,1.0,0,0,TERRENO,0,TERRENO,NaN,NaN
4,13944000322,AV NORDESTINA,3010,AP 305 SUB 1 BL 1,VL CURUCA,NaN,8032000.0,1.Compra e venda,200550.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,200550.0,1.Sistema Financeiro de Habitação,100433.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0,NaN


In [7]:
# removendo a última coluna que está vazia
df_combined = df_combined.drop(columns='Unnamed: 28')
df_combined.head()

,N° do Cadastro (SQL),Nome do Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza de Transação,Valor de Transação (declarado pelo contribuinte),Data de Transação,Valor Venal de Referência,Proporção Transmitida (%),Valor Venal de Referência (proporcional),Base de Cálculo adotada,Tipo de Financiamento,Valor Financiado,Cartório de Registro,Matrícula do Imóvel,Situação do SQL,Área do Terreno (m2),Testada (m),Fração Ideal,Área Construída (m2),Uso (IPTU),Descrição do uso (IPTU),Padrão (IPTU),Descrição do padrão (IPTU),ACC (IPTU)
0,13944000322,AV NORDESTINA,3010,AP 918 TORRE 2,VL CURUCA,NaN,8032000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,15828870.0,0.20,31657.7400,264000.0,1.Sistema Financeiro de Habitação,166320.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
1,6726500084,R JURIPIRANGA,70,ANT 16,PARQUE VITORIA,NaN,2269090.0,1.Compra e venda,800000.0,2022-12-23 00:00:00,846885.0,100.00,846885.0000,846885.0,1.Sistema Financeiro de Habitação,400000.00,15º Cartório de Registro de Imóvel,87025,Ativo Predial,210,10.0,1.0,277,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,2009.0
2,13944000322,AV NORDESTINA,3010,AP 504 SUB 2 BL 2,VL CURUCA,NaN,8032000.0,1.Compra e venda,203700.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,203700.0,1.Sistema Financeiro de Habitação,144059.51,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
3,5614900926,R ANTONIO DE BARROS,99999,AP 614 BL 3,TATUAPE,NaN,3401001.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.00,9º Cartório de Registro de Imóvel,336376,Ativo Territorial,1000,20.0,1.0,0,0,TERRENO,0,TERRENO,NaN
4,13944000322,AV NORDESTINA,3010,AP 305 SUB 1 BL 1,VL CURUCA,NaN,8032000.0,1.Compra e venda,200550.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,200550.0,1.Sistema Financeiro de Habitação,100433.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0


Podemos ver que a coluna 'CEP' é do tipo float que não é a melhor escolha para esse tipo de dado, pois os ceps que começam com zero tem o primeiro caratere removido.
Então precisamos mudar o tipo para string e adicionar os zeros à esqueda que foram removidos

In [8]:
# verificando os tipos de dados por coluna:
df_combined.dtypes

N° do Cadastro (SQL)                                  int64
Nome do Logradouro                                   object
Número                                                int64
Complemento                                          object
Bairro                                               object
Referência                                           object
CEP                                                 float64
Natureza de Transação                                object
Valor de Transação (declarado pelo contribuinte)     object
Data de Transação                                    object
Valor Venal de Referência                           float64
Proporção Transmitida (%)                           float64
Valor Venal de Referência (proporcional)            float64
Base de Cálculo adotada                             float64
Tipo de Financiamento                                object
Valor Financiado                                    float64
Cartório de Registro                    

In [9]:
# remove as linhas em que o CEP é nulo
df_combined = df_combined.dropna(subset=['CEP'])

# transforma a coluna CEP em int para remover o ponto flutuante
df_combined['CEP'] = df_combined['CEP'].astype(int)

# Converter a coluna CEP para string
df_combined['CEP'] = df_combined['CEP'].astype(str)

# Adicionar zero à esquerda se o valor tiver 7 caracteres
df_combined['CEP'] = df_combined['CEP'].apply(lambda x: x.zfill(8) if len(x) == 7 else x)

df_combined['CEP']

0         08032000
1         02269090
2         08032000
3         03401001
4         08032000
            ...   
175454    05304000
175455    04363060
175456    04065050
175457    02132120
175458    04671072
Name: CEP, Length: 175457, dtype: object

# Data Enrichment:

### Conectando o CEP a uma API para obter dados corretos de endereços.

Encontrei uma API bacana no githuhb para coletar os dados de endereço com base no CEP.

Vou usar esse método pois é bom para organizar os dados de endereço que nem sempre são padronizados, por vezes encontramos variações e abreviações nos nomes de bairros (Ex. Vila Mariana, Vl. Mariana, V. Mariana, etc) e é isso que quero evitar.

A API ultilizada está disponibilizada no seguinte link: https://github.com/raniellyferreira/awesomeapi-cep

In [10]:
#  função para pegar os dados de endereço usando o CEP

def get_cep_info(cep):
  url = f'https://cep.awesomeapi.com.br/json/{cep}'
  response = requests.get(url)
  if response.status_code == 200:
      return json.loads(response.content)
  else:
      print(f"Falha ao buscar informações para o CEP {cep}, Status code: {response.status_code}")
      return None

In [11]:
# exemplo:
get_cep_info('08032000')

{'cep': '08032000',
 'address_type': 'Avenida',
 'address_name': 'Nordestina',
 'address': 'Avenida Nordestina',
 'state': 'SP',
 'district': 'Vila Nova Curuçá',
 'lat': '-23.51802',
 'lng': '-46.42164',
 'city': 'São Paulo',
 'city_ibge': '3550308',
 'ddd': '11'}

In [12]:
# criando um array com os valores unicos da coluna CEP
unique_ceps = df_combined['CEP'].unique()
unique_ceps

array(['08032000', '02269090', '03401001', ..., '01421902', '03814080',
       '01540060'], dtype=object)

In [13]:
# cria uma lista vazia para adicionar os dados dos CEPs
cep_data = []

# aplica a funcão get_cep_info e mostra uma barra de carregamento usando a biblioteca tqdm
for cep in tqdm(unique_ceps, desc="Processando CEPs"):
    info = get_cep_info(cep)
    if info is not None:
        cep_data.append(info)

Processando CEPs:   0%|          | 0/20083 [00:00<?, ?it/s]

Processando CEPs:   7%|▋         | 1328/20083 [01:26<15:08, 20.65it/s]  

Falha ao buscar informações para o CEP 05061901, Status code: 404


Processando CEPs:  10%|█         | 2081/20083 [02:03<12:44, 23.54it/s]

Falha ao buscar informações para o CEP 05218900, Status code: 404


Processando CEPs:  21%|██        | 4177/20083 [03:43<13:12, 20.08it/s]

Falha ao buscar informações para o CEP 04785165, Status code: 404


Processando CEPs: 100%|██████████| 20083/20083 [17:09<00:00, 19.50it/s]  


In [14]:
# transforma a lista cep_data em um dataframe do pandas
df_cep_info = pd.DataFrame(cep_data)
df_cep_info

,cep,address_type,address_name,address,state,district,lat,lng,city,city_ibge,ddd
0,08032000,Avenida,Nordestina,Avenida Nordestina,SP,Vila Nova Curuçá,-23.51802,-46.42164,São Paulo,3550308,11
1,02269090,Rua,Juripiranga,Rua Juripiranga,SP,Parque Vitória,-23.47005,-46.59494,São Paulo,3550308,11
2,03401001,Rua,Antônio de Barros,Rua Antônio de Barros,SP,Vila Carrão,-23.53885,-46.55771,São Paulo,3550308,11
3,02938000,Avenida,Raimundo Pereira de Magalhães,Avenida Raimundo Pereira de Magalhães,SP,Pirituba,-23.47948,-46.72676,São Paulo,3550308,11
4,05535000,Rua,Professor Máximo Ribeiro Nunes,Rua Professor Máximo Ribeiro Nunes,SP,Jardim Peri Peri,-23.58464,-46.73714,São Paulo,3550308,11
...,...,...,...,...,...,...,...,...,...,...,...
20075,08150090,Rua,José Eleutério dos Santos,Rua José Eleutério dos Santos,SP,Parque Santa Rita,-23.51453,-46.41556,São Paulo,3550308,11
20076,04021001,Rua,Sena Madureira,Rua Sena Madureira,SP,Vila Clementino,-23.59386,-46.64467,São Paulo,3550308,11
20077,01421902,Alameda,"Itu, 1437","Alameda Itu, 1437",SP,Cerqueira César,-23.55894,-46.66539,São Paulo,3550308,11
20078,03814080,Rua,Reverendo João Euclides Pereira,Rua Reverendo João Euclides Pereira,SP,Jardim Matarazzo,-23.48761,-46.48105,São Paulo,3550308,11


### Um outro adendo aos dados é a oreganização dos CEPs por zona de acordo com os dados da Prefeitura de São Paulo, para fazer isso vou utilizar apenas os 3 primeiros digitos de cada CEP para identificar a zona correspondente:

In [15]:
zonas = {
    'centro': ['010', '011', '012', '013', '014', '015'],
    'norte': ['020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '051', '052'],
    'leste': ['030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '080', '081', '082', '083', '084'],
    'sul': ['040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '057', '058'],
    'oeste': ['050', '051', '052', '053', '054', '055', '056']
}

In [16]:
# função que mapeia as zonas:
def zone_mapping(cep):
    prefixo_cep = str(cep)[:3]
    for zone, ceps in zonas.items():
        if prefixo_cep in ceps:
            return zone
    return 'desconhecida'

In [17]:
# aplicando a função mapear_zona no df_cep_info:
df_cep_info['zone'] = df_cep_info['cep'].apply(zone_mapping)

In [18]:
df_cep_info.head()

,cep,address_type,address_name,address,state,district,lat,lng,city,city_ibge,ddd,zone
0,08032000,Avenida,Nordestina,Avenida Nordestina,SP,Vila Nova Curuçá,-23.51802,-46.42164,São Paulo,3550308,11,leste
1,02269090,Rua,Juripiranga,Rua Juripiranga,SP,Parque Vitória,-23.47005,-46.59494,São Paulo,3550308,11,norte
2,03401001,Rua,Antônio de Barros,Rua Antônio de Barros,SP,Vila Carrão,-23.53885,-46.55771,São Paulo,3550308,11,leste
3,02938000,Avenida,Raimundo Pereira de Magalhães,Avenida Raimundo Pereira de Magalhães,SP,Pirituba,-23.47948,-46.72676,São Paulo,3550308,11,norte
4,05535000,Rua,Professor Máximo Ribeiro Nunes,Rua Professor Máximo Ribeiro Nunes,SP,Jardim Peri Peri,-23.58464,-46.73714,São Paulo,3550308,11,oeste


### Vou remover algumas colunas e renomear as que ficaram

In [19]:
df_combined = df_combined.drop(columns=['Nome do Logradouro', 'Complemento',
       'Bairro', 'Referência'])

In [20]:
names = {
    'N° do Cadastro (SQL)': 'sql_registration_number',
    'Número': 'number', 
    'CEP': 'cep', 
    'Natureza de Transação': 'transaction_nature',
    'Valor de Transação (declarado pelo contribuinte)': 'transaction_value_declared', 
    'Data de Transação': 'transaction_date',
    'Valor Venal de Referência': 'reference_market_value', 
    'Proporção Transmitida (%)': 'transmitted_proportion',
    'Valor Venal de Referência (proporcional)': 'proportional_reference_market_value', 
    'Base de Cálculo adotada': 'adopted_calculation_base',
    'Tipo de Financiamento': 'financing_type', 
    'Valor Financiado': 'financed_value', 
    'Cartório de Registro': 'registry_office', 
    'Matrícula do Imóvel': 'property_registration', 
    'Situação do SQL': 'sql_status', 
    'Área do Terreno (m2)': 'land_area_m2',
    'Testada (m)': 'frontage_m', 
    'Fração Ideal': 'ideal_fraction', 
    'Área Construída (m2)': 'built_area_m2', 
    'Uso (IPTU)': 'iptu_use',
    'Descrição do uso (IPTU)': 'iptu_use_description', 
    'Padrão (IPTU)': 'iptu_standard',
    'Descrição do padrão (IPTU)': 'iptu_standard_description', 
    'ACC (IPTU)': 'iptu_acc'
}

df_combined.rename(columns=names, inplace=True)


In [21]:
df_combined.head()

,sql_registration_number,number,cep,transaction_nature,transaction_value_declared,transaction_date,reference_market_value,transmitted_proportion,proportional_reference_market_value,adopted_calculation_base,financing_type,financed_value,registry_office,property_registration,sql_status,land_area_m2,frontage_m,ideal_fraction,built_area_m2,iptu_use,iptu_use_description,iptu_standard,iptu_standard_description,iptu_acc
0,13944000322,3010,08032000,1.Compra e venda,264000.0,2022-12-22 00:00:00,15828870.0,0.20,31657.7400,264000.0,1.Sistema Financeiro de Habitação,166320.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
1,6726500084,70,02269090,1.Compra e venda,800000.0,2022-12-23 00:00:00,846885.0,100.00,846885.0000,846885.0,1.Sistema Financeiro de Habitação,400000.00,15º Cartório de Registro de Imóvel,87025,Ativo Predial,210,10.0,1.0,277,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,2009.0
2,13944000322,3010,08032000,1.Compra e venda,203700.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,203700.0,1.Sistema Financeiro de Habitação,144059.51,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
3,5614900926,99999,03401001,1.Compra e venda,264000.0,2022-12-22 00:00:00,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.00,9º Cartório de Registro de Imóvel,336376,Ativo Territorial,1000,20.0,1.0,0,0,TERRENO,0,TERRENO,NaN
4,13944000322,3010,08032000,1.Compra e venda,200550.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,200550.0,1.Sistema Financeiro de Habitação,100433.00,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0


# Juntando os dados:

In [22]:
# "mergeia" os dois dataframes usando a coluna cep
df_final = pd.merge(df_combined, df_cep_info, on='cep', how='left')
df_final.columns

Index(['sql_registration_number', 'number', 'cep', 'transaction_nature',
       'transaction_value_declared', 'transaction_date',
       'reference_market_value', 'transmitted_proportion',
       'proportional_reference_market_value', 'adopted_calculation_base',
       'financing_type', 'financed_value', 'registry_office',
       'property_registration', 'sql_status', 'land_area_m2', 'frontage_m',
       'ideal_fraction', 'built_area_m2', 'iptu_use', 'iptu_use_description',
       'iptu_standard', 'iptu_standard_description', 'iptu_acc',
       'address_type', 'address_name', 'address', 'state', 'district', 'lat',
       'lng', 'city', 'city_ibge', 'ddd', 'zone'],
      dtype='object')

### Para finalizar vou reordenar as colunas

In [23]:
# Lista com a ordem desejada das colunas
colunas_ordenadas = [
    'sql_registration_number', 'transaction_nature', 'transaction_value_declared', 
    'transaction_date', 'reference_market_value', 'transmitted_proportion', 
    'proportional_reference_market_value', 'adopted_calculation_base', 
    'financing_type', 'financed_value', 'registry_office', 
    'property_registration', 'sql_status', 'land_area_m2', 'frontage_m', 
    'ideal_fraction', 'built_area_m2', 'iptu_use', 'iptu_use_description',
    'iptu_standard', 'iptu_standard_description', 
    'iptu_acc', 'cep', 'number', 'address_type', 'address_name', 
    'address', 'state', 'district', 'zone', 'lat', 'lng', 'city', 'city_ibge', 'ddd'
]

# Reordena o DataFrame de acordo com a lista de colunas
df_final = df_final[colunas_ordenadas]

In [24]:
df_final.tail()

,sql_registration_number,transaction_nature,transaction_value_declared,transaction_date,reference_market_value,transmitted_proportion,proportional_reference_market_value,adopted_calculation_base,financing_type,financed_value,registry_office,property_registration,sql_status,land_area_m2,frontage_m,ideal_fraction,built_area_m2,iptu_use,iptu_use_description,iptu_standard,iptu_standard_description,iptu_acc,cep,number,address_type,address_name,address,state,district,zone,lat,lng,city,city_ibge,ddd
175452,8004200672,1.Compra e venda,730000.0,2023-12-28 00:00:00,993292.0,100.0,993292.0,993292.0,1.Sistema Financeiro de Habitação,550000.0,10º Cartório de Registro de Imóvel,10553,Ativo Predial,175,7.00,1.0000,160,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,1970.0,05304000,174,Rua,Frederico Wolf,Rua Frederico Wolf,SP,Vila Leopoldina,oeste,-23.5246,-46.7353,São Paulo,3550308,11
175453,8911202835,1.Compra e venda,282000.0,2023-12-29 00:00:00,564787.0,50.0,282393.5,282393.5,NaN,0.0,8º Cartório de Registro de Imóvel,127217,Ativo Predial,1200,40.00,0.0167,154,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,1997.0,04363060,31,Rua,Arapá,Rua Arapá,SP,Vila Mascote,sul,-23.64732,-46.66704,São Paulo,3550308,11
175454,4702500198,1.Compra e venda,745000.0,2023-12-28 00:00:00,301382.0,100.0,301382.0,745000.0,1.Sistema Financeiro de Habitação,670500.0,14º Cartório de Registro de Imóvel,65263,Ativo Predial,76,4.00,1.0000,135,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,1947.0,04065050,253,Alameda,dos Piratinins,Alameda dos Piratinins,SP,Planalto Paulista,sul,-23.61683,-46.64648,São Paulo,3550308,11
175455,6503501323,1.Compra e venda,490000.0,2024-01-03 00:00:00,463297.0,100.0,463297.0,490000.0,NaN,0.0,17º Cartório de Registro de Imóvel,73374,Ativo Predial,224,6.20,1.0000,120,14,RESIDÊNCIA E OUTRO USO (PREDOMINÂNCIA RESIDENC...,12,RESIDENCIAL HORIZONTAL,1978.0,02132120,453,Rua,Yamato,Rua Yamato,SP,Jardim Japão,norte,-23.50092,-46.57553,São Paulo,3550308,11
175456,9046300811,1.Compra e venda,193000.0,2008-08-25 00:00:00,145832.0,100.0,145832.0,193000.0,NaN,0.0,11º Cartório de Registro de Imóvel,241626,Ativo Predial,11672,109.47,0.0034,163,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,1990.0,04671072,853,Rua,Sócrates,Rua Sócrates,SP,Vila Sofia,sul,-23.65962,-46.6918,São Paulo,3550308,11


In [25]:
df_final = df_final.astype(str)
df_final.to_csv('dados.csv', index=False)